In [1]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile

os.chdir('F:\\Codigo\\Python\\TETIS')
from funciones_exportar_TETIS import export_heading, export_series, export_control
from funciones_resultados_TETIS import leer_caudal, leer_SCEUA, NSE, sesgo, RMSE, KGE

In [ ]:
def crear_VarSCEUA(par_cal, ruta, flow, met, start=1, OF=6, p_forma=2.0, n_dias=30, A_pond=0):
    """Crea el archivo de configuración de la calibración automática del módulo hidrológico de TETIS.
    
    Entradas:
    ---------
    par_cal: list. Nombre de los parámetros a calibrar
    ruta:    string. Carpeta en la que guardar el archivo
    flow:    string. Tipo de caudal al que se refieren los valores de la calibración. Sólo influye en el nombre del archivo exportado
    met:     string. Método de calibración empleado en la optimización de los parámetros. Sólo influye en el nombre del arcihvo exportado
    start:   integer. Día de la simulación a partir del que calcular la función objetivo
    OF:      integer. Función objetivo a emplear: 4-RMSE, 5-volumen acumulado, 6-NSE
    p_forma: float. Valor del parámetro de forma para el cálculo de la función objetivo HMLE
    n_dias:  integer. Número de días en los que agrupar el caudal para el cálculo de la función objetivo RMSE mensual
    A_pond:  integer. Si ponderar (1) o no (0) la función objetivo según el área de la cuenca de los puntos aforados
    
    Salidas:
    --------
    Archivo 'Var-SCEUA_flow_met.txt'"""
    
    # TABLA POR DEFECTO DE RANGOS Y VALOR INICIAL DE LOS PARÁMETROS
    # -------------------------------------------------------------
    # definir valores mínimos, máximos y por defecto
    mins = [0.01, 0, 0, 0.001, 0, 0.001, 0, 0.001, 0, 0, 0, 0, 0, 0, -0.2, 0, 0, 1, 3, -2, 0, 0]
    maxs = [3, 2, 1.5, 10, 1.5, 5000, 1, 50000, 1.5, 100, 100, 100, 100, 100, 0.2, 1, 1, 3, 6, 4, 3, 1000000]
    defs = [1, 1, 0.2, 1, 0.4, 10, 0.5, 200, 1, 50, 0.5, 0.5, 0.5, 50, 0, 1, 1, 1, 1, 1, 0, 100000]
    
    # definir el nombre de los parámetros calibrables
    # factores correctores
    idx1 = ['FC' + str(i) for i in range(1, 10)]
    # estados iniciales
    idx2 = ['H' + str(i) for i in range(1, 6)]
    # otras variables
    idx3 = ['Bppt', 'Bnieve', 'FC0', 'DDF1', 'DDF2', 'Tb', 'p1', 'H3max']
    idx = idx1 + idx2 + idx3
    
    # generar data frame
    VarSCEUA = pd.DataFrame(data=[mins, maxs, defs], columns=idx).T
    VarSCEUA.columns = ['min', 'max', 'init']
    VarSCEUA['cal'] = 'F'
    VarSCEUA.loc[par_cal, 'cal'] = 'T'
    
    # EXPORTAR ARCHIVO 'Var-SCEUA.txt'
    # --------------------------------
    filename = 'Var-SCEUA_' + flow + '_' + met + '.txt'
    with open(ruta + filename, 'w+') as f:
        # Exportar variables de cálculo de la función objetivo
        f.write(str(start) + '\n')
        f.write(str(OF) + '\n')
        f.write(str(round(p_forma, 2)) + '\n')
        f.write(str(n_dias) + '\n')
        if A_pond not in [0, 1]:
            print('ERROR. "A_pond" debe ser 0 ó 1')
            return
        else:
            f.write(str(A_pond) + '\n')
        
        # Exportar la configuración del espacio de parámetros a analizar
        for FC in VarSCEUA.index:
            val = VarSCEUA.loc[FC,:]
            f.write('   {0:7.5f}   {1:7.5f}   {2:7.5f}      {3:1}\n'.format(val[0], val[1], val[2], val[3]))

In [20]:
par_cal = ['FC3', 'FC4', 'FC5', 'FC6']
ruta = 'C:/Users/casadoj/Desktop/BLF_MAD-0.4/'
flow = 'quick'
met = 'met2'
start=365
OF=6
p_forma=2
n_dias=30
A_pond=0

In [58]:
    # TABLA POR DEFECTO DE RANGOS Y VALOR INICIAL DE LOS PARÁMETROS
    # -------------------------------------------------------------
    # definir valores mínimos, máximos y por defecto
    mins = [0.01, 0, 0, 0.001, 0, 0.001, 0, 0.001, 0, 0, 0, 0, 0, 0, -0.2, 0, 0, 1, 3, -2, 0, 0]
    maxs = [3, 2, 1.5, 10, 1.5, 5000, 1, 50000, 1.5, 100, 100, 100, 100, 100, 0.2, 1, 1, 3, 6, 4, 3, 1000000]
    defs = [1, 1, 0.2, 1, 0.4, 10, 0.5, 200, 1, 50, 0.5, 0.5, 0.5, 50, 0, 1, 1, 1, 1, 1, 0, 100000]
    
    # definir el nombre de los parámetros calibrables
    # factores correctores
    idx1 = ['FC' + str(i) for i in range(1, 10)]
    # estados iniciales
    idx2 = ['H' + str(i) for i in range(1, 6)]
    # otras variables
    idx3 = ['Bppt', 'Bnieve', 'FC0', 'DDF1', 'DDF2', 'Tb', 'p1', 'H3max']
    idx = idx1 + idx2 + idx3
    
    # generar data frame
    VarSCEUA = pd.DataFrame(data=[mins, maxs, defs], columns=idx).T
    VarSCEUA.columns = ['min', 'max', 'init']
    VarSCEUA['cal'] = 'F'
    VarSCEUA.loc[par_cal, 'cal'] = 'T'
    

In [59]:
for c in VarSCEUA.columns[:-1]:
    for r in VarSCEUA.index:
        val = str(VarSCEUA.loc[r, c])
        while len(val) < 7:
            val += '0'
        if len(val) >= 7:
            val = val[:7]
        VarSCEUA.loc[r, c] = val
VarSCEUA

,min,max,init,cal
FC1,0.01000,3.00000,1.00000,F
FC2,0.00000,2.00000,1.00000,F
FC3,0.00000,1.50000,0.20000,T
FC4,0.00100,10.0000,1.00000,T
FC5,0.00000,1.50000,0.40000,T
FC6,0.00100,5000.00,10.0000,T
FC7,0.00000,1.00000,0.50000,F
FC8,0.00100,50000.0,200.000,F
FC9,0.00000,1.50000,1.00000,F
H1,0.00000,100.000,50.0000,F


In [60]:
    # EXPORTAR ARCHIVO 'Var-SCEUA.txt'
    # --------------------------------
    filename = 'Var-SCEUA_' + flow + '_' + met + '.txt'
    with open(ruta + filename, 'w') as f:
        # Exportar variables de cálculo de la función objetivo
        f.write(str(start) + '\n')
        f.write(str(OF) + '\n')
        f.write(str(round(float(p_forma), 3)) + '\n')
        f.write(str(n_dias) + '\n')
        if A_pond not in [0, 1]:
            print('ERROR. "A_pond" debe ser 0 ó 1')
            #return
        else:
            f.write(str(A_pond) + '\n')
        
        # Exportar la configuración del espacio de parámetros a analizar
        for FC in VarSCEUA.index:
            val = VarSCEUA.loc[FC,:]
            f.write('   {0:7}   {1:7}   {2:7}      {3:1}\n'.format(val[0], val[1], val[2], val[3]))

In [30]:
[str(round(val, 5))[:7] for val in VarSCEUA['min']]

['0.01',
 '0.0',
 '0.0',
 '0.001',
 '0.0',
 '0.001',
 '0.0',
 '0.001',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '-0.2',
 '0.0',
 '0.0',
 '1.0',
 '3.0',
 '-2.0',
 '0.0',
 '0.0']

In [35]:
val = 0.001
val = str(val)
while len(val) <= 7:
    val += '0'
val

'0.001000'

In [34]:
len(val)

5

In [61]:
VarSCEUA

,min,max,init,cal
FC1,0.01000,3.00000,1.00000,F
FC2,0.00000,2.00000,1.00000,F
FC3,0.00000,1.50000,0.20000,T
FC4,0.00100,10.0000,1.00000,T
FC5,0.00000,1.50000,0.40000,T
FC6,0.00100,5000.00,10.0000,T
FC7,0.00000,1.00000,0.50000,F
FC8,0.00100,50000.0,200.000,F
FC9,0.00000,1.50000,1.00000,F
H1,0.00000,100.000,50.0000,F
